# Fintech Sector Portfolio Analysis

(Insert project description)

(Insert table of contents with clickable anchor links)

In [1]:
import pandas as pd
from pathlib import Path
from MCForecastTools import MCSimulation
import yfinance as yf

## Collect and Clean Data

In [2]:
def get_daily_returns_df(yf_tickers, ticker_names, period):
    # create pandas DataFrames for each stock in the yf ticker object, based on provided time period
    t1_df = yf_tickers.tickers[ticker_names[0]].history(period=period)
    t2_df = yf_tickers.tickers[ticker_names[1]].history(period=period)
    t3_df = yf_tickers.tickers[ticker_names[2]].history(period=period)

    # print out each DataFrame to verify data collection 
    print(f'{ticker_names[0]} DataFrame')
    display(t1_df.head())
    display(t1_df.tail())

    print(f'{ticker_names[1]} DataFrame')
    display(t2_df.head())
    display(t2_df.tail())

    print(f'{ticker_names[2]} DataFrame')
    display(t3_df.head())
    display(t3_df.tail())
    print('\n')

    # select only the close column from the DataFrames 
    t1_df = t1_df[['Close']]
    t2_df = t2_df[['Close']]
    t3_df = t3_df[['Close']]

    # concat close data into one DataFrame 
    daily_returns_df = pd.concat([t1_df, t2_df, t3_df], axis=1)

    # rename columns to each stock ticker instead of just "Close"
    daily_returns_df.columns = ticker_names

    # print out the resulting DataFrame to verify correct closing price data
    print(f'Closing price DataFrame for {ticker_names[0]}, {ticker_names[1]}, {ticker_names[2]}')
    display(daily_returns_df.head())
    display(daily_returns_df.tail())
    print('\n')

    # convert closing prices to daily returns 
    daily_returns_df = daily_returns_df.pct_change().dropna()

    # print final daily returns DataFrame
    print(f'Daily returns DataFrame for {ticker_names[0]}, {ticker_names[1]}, {ticker_names[2]}')
    display(daily_returns_df.head())
    display(daily_returns_df.tail())

    return daily_returns_df

In [3]:
# create yfinance ticker object for paytech stocks
paytech_tickers = yf.Tickers('pypl sq ma')

# use tickers and 5 year period to get 5 years of paytech stock data 
paytech_daily_returns =  get_daily_returns_df(paytech_tickers, ['PYPL', 'SQ', 'MA'], '5y')

PYPL DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,75.589996,75.744003,68.610001,70.970001,26283600,0,0
2017-12-05 00:00:00-05:00,69.809998,72.750000,69.610001,71.199997,16634000,0,0
2017-12-06 00:00:00-05:00,70.889999,73.199997,70.830002,73.139999,9331400,0,0
2017-12-07 00:00:00-05:00,73.250000,74.069000,73.250000,73.690002,7937700,0,0
2017-12-08 00:00:00-05:00,74.709999,74.709999,72.599998,72.910004,8350400,0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,79.930000,82.919998,79.639999,79.930000,11282600,0,0
2022-11-29 00:00:00-05:00,79.230003,80.750000,77.260002,77.639999,15491100,0,0
2022-11-30 00:00:00-05:00,77.699997,79.110001,75.320000,78.410004,25747300,0,0
2022-12-01 00:00:00-05:00,79.110001,80.746002,78.139999,78.529999,15189000,0,0
2022-12-02 00:00:00-05:00,76.820000,77.059998,74.519997,74.660004,24212900,0,0


SQ DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,38.410000,38.689999,35.900002,36.869999,29566900,0,0
2017-12-05 00:00:00-05:00,36.750000,38.590000,36.740002,37.599998,21276200,0,0
2017-12-06 00:00:00-05:00,37.730000,39.770000,37.270000,37.610001,28820400,0,0
2017-12-07 00:00:00-05:00,39.325001,39.459999,37.959999,38.730000,23721600,0,0
2017-12-08 00:00:00-05:00,39.610001,39.790001,37.939999,38.090000,20917800,0,0


...

,PYPL,SQ,MA
Date,,,
2017-12-04 00:00:00-05:00,70.970001,36.869999,139.599030
2017-12-05 00:00:00-05:00,71.199997,37.599998,141.253647
2017-12-06 00:00:00-05:00,73.139999,37.610001,143.482452
2017-12-07 00:00:00-05:00,73.690002,38.730000,145.458267
2017-12-08 00:00:00-05:00,72.910004,38.090000,145.886505


,PYPL,SQ,MA
Date,,,
2022-11-28 00:00:00-05:00,79.930000,61.369999,344.589996
2022-11-29 00:00:00-05:00,77.639999,62.110001,343.130005
2022-11-30 00:00:00-05:00,78.410004,67.769997,356.399994
2022-12-01 00:00:00-05:00,78.529999,69.169998,360.799988
2022-12-02 00:00:00-05:00,74.660004,68.180000,360.059998




Daily returns DataFrame for PYPL, SQ, MA


,PYPL,SQ,MA
Date,,,
2017-12-05 00:00:00-05:00,0.003241,0.019799,0.011853
2017-12-06 00:00:00-05:00,0.027247,0.000266,0.015779
2017-12-07 00:00:00-05:00,0.007520,0.029779,0.013770
2017-12-08 00:00:00-05:00,-0.010585,-0.016525,0.002944
2017-12-11 00:00:00-05:00,0.005212,0.005513,0.006605


,PYPL,SQ,MA
Date,,,
2022-11-28 00:00:00-05:00,-0.001873,-0.031714,-0.019073
2022-11-29 00:00:00-05:00,-0.028650,0.012058,-0.004237
2022-11-30 00:00:00-05:00,0.009918,0.091129,0.038673
2022-12-01 00:00:00-05:00,0.001530,0.020658,0.012346
2022-12-02 00:00:00-05:00,-0.049280,-0.014313,-0.002051


In [4]:
# create yfinance ticker object for lending stocks
lending_tickers = yf.Tickers('tree lc fis')

# use tickers and 5 year period to get 5 years of paytech stock data 
lending_daily_returns =  get_daily_returns_df(lending_tickers, ['TREE', 'LC', 'FIS'], '5y')

TREE DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,309.850006,316.250000,301.450012,303.399994,246900,0,0
2017-12-05 00:00:00-05:00,301.950012,313.299988,298.609985,305.200012,185000,0,0
2017-12-06 00:00:00-05:00,305.350006,310.799988,304.200012,306.899994,190300,0,0
2017-12-07 00:00:00-05:00,307.049988,316.600006,305.980011,311.250000,125300,0,0
2017-12-08 00:00:00-05:00,314.950012,319.429993,308.899994,315.049988,258700,0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,23.379999,23.379999,22.200001,22.370001,235900,0,0
2022-11-29 00:00:00-05:00,22.500000,23.459999,22.010000,23.330000,262400,0,0
2022-11-30 00:00:00-05:00,23.200001,23.990000,21.750000,23.950001,376000,0,0
2022-12-01 00:00:00-05:00,24.000000,25.240000,23.170000,25.120001,256900,0,0
2022-12-02 00:00:00-05:00,24.250000,25.110001,23.629999,24.980000,204100,0,0


LC DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,21.950001,22.275000,21.350000,21.850000,1400220,0,0.0
2017-12-05 00:00:00-05:00,21.850000,21.950001,21.250000,21.250000,1117300,0,0.0
2017-12-06 00:00:00-05:00,21.150000,21.400000,20.799999,21.250000,1079140,0,0.0
2017-12-07 00:00:00-05:00,18.549999,19.700001,16.450001,17.950001,9593000,0,0.0
2017-12-08 00:00:00-05:00,18.700001,18.700001,17.299999,17.549999,4030720,0,0.0


...

,TREE,LC,FIS
Date,,,
2017-12-04 00:00:00-05:00,303.399994,21.850000,87.857101
2017-12-05 00:00:00-05:00,305.200012,21.250000,89.302132
2017-12-06 00:00:00-05:00,306.899994,21.250000,89.424126
2017-12-07 00:00:00-05:00,311.250000,17.950001,89.959000
2017-12-08 00:00:00-05:00,315.049988,17.549999,89.940216


,TREE,LC,FIS
Date,,,
2022-11-28 00:00:00-05:00,22.370001,10.06,65.529999
2022-11-29 00:00:00-05:00,23.330000,10.26,67.529999
2022-11-30 00:00:00-05:00,23.950001,10.31,72.580002
2022-12-01 00:00:00-05:00,25.120001,10.24,73.180000
2022-12-02 00:00:00-05:00,24.980000,10.18,74.059998




Daily returns DataFrame for TREE, LC, FIS


,TREE,LC,FIS
Date,,,
2017-12-05 00:00:00-05:00,0.005933,-0.027460,0.016448
2017-12-06 00:00:00-05:00,0.005570,0.000000,0.001366
2017-12-07 00:00:00-05:00,0.014174,-0.155294,0.005981
2017-12-08 00:00:00-05:00,0.012209,-0.022284,-0.000209
2017-12-11 00:00:00-05:00,-0.019521,0.108262,-0.001461


,TREE,LC,FIS
Date,,,
2022-11-28 00:00:00-05:00,-0.051717,-0.047348,-0.006820
2022-11-29 00:00:00-05:00,0.042915,0.019881,0.030520
2022-11-30 00:00:00-05:00,0.026575,0.004873,0.074782
2022-12-01 00:00:00-05:00,0.048852,-0.006790,0.008267
2022-12-02 00:00:00-05:00,-0.005573,-0.005859,0.012025


### Get 5 years of data for Paytech sector - PayPal, Square, and MasterCard

In [5]:
# create yfinance ticker object for paytech stocks
paytech_tickers = yf.Tickers('pypl sq ma')

In [6]:
# create pandas DataFrames for 5 years worth of data for each stock 
pypl_df = paytech_tickers.tickers['PYPL'].history(period='5y')
sq_df = paytech_tickers.tickers['SQ'].history(period='5y')
ma_df = paytech_tickers.tickers['MA'].history(period='5y')

print('PayPal DataFrame')
display(pypl_df.head())
display(pypl_df.tail())

print('Square DataFrame')
display(sq_df.head())
display(sq_df.tail())

print('MasterCard DataFrame')
display(ma_df.head())
display(ma_df.tail())

PayPal DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,75.589996,75.744003,68.610001,70.970001,26283600,0,0
2017-12-05 00:00:00-05:00,69.809998,72.750000,69.610001,71.199997,16634000,0,0
2017-12-06 00:00:00-05:00,70.889999,73.199997,70.830002,73.139999,9331400,0,0
2017-12-07 00:00:00-05:00,73.250000,74.069000,73.250000,73.690002,7937700,0,0
2017-12-08 00:00:00-05:00,74.709999,74.709999,72.599998,72.910004,8350400,0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,79.930000,82.919998,79.639999,79.930000,11282600,0,0
2022-11-29 00:00:00-05:00,79.230003,80.750000,77.260002,77.639999,15491100,0,0
2022-11-30 00:00:00-05:00,77.699997,79.110001,75.320000,78.410004,25747300,0,0
2022-12-01 00:00:00-05:00,79.110001,80.746002,78.139999,78.529999,15189000,0,0
2022-12-02 00:00:00-05:00,76.820000,77.059998,74.519997,74.660004,24212900,0,0


Square DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,38.410000,38.689999,35.900002,36.869999,29566900,0,0
2017-12-05 00:00:00-05:00,36.750000,38.590000,36.740002,37.599998,21276200,0,0
2017-12-06 00:00:00-05:00,37.730000,39.770000,37.270000,37.610001,28820400,0,0
2017-12-07 00:00:00-05:00,39.325001,39.459999,37.959999,38.730000,23721600,0,0
2017-12-08 00:00:00-05:00,39.610001,39.790001,37.939999,38.090000,20917800,0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,61.910000,63.830002,61.119999,61.369999,7239900,0,0
2022-11-29 00:00:00-05:00,62.070000,63.889999,61.730000,62.110001,8219900,0,0
2022-11-30 00:00:00-05:00,62.099998,68.000000,61.852001,67.769997,14298600,0,0
2022-12-01 00:00:00-05:00,68.019997,69.239998,66.660004,69.169998,9430000,0,0
2022-12-02 00:00:00-05:00,67.269997,68.940002,66.620003,68.180000,7716000,0,0


MasterCard DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,145.915689,146.567792,139.550377,139.599030,7490100,0.0,0
2017-12-05 00:00:00-05:00,139.861819,142.411849,136.854355,141.253632,6807100,0.0,0
2017-12-06 00:00:00-05:00,141.195259,144.115131,140.757267,143.482483,3752000,0.0,0
2017-12-07 00:00:00-05:00,143.570036,146.392587,143.570036,145.458221,3417900,0.0,0
2017-12-08 00:00:00-05:00,146.402333,146.421803,145.185720,145.886490,2521500,0.0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,346.679993,350.230011,343.230011,344.589996,2226200,0.0,0
2022-11-29 00:00:00-05:00,343.649994,344.720001,340.750000,343.130005,1724200,0.0,0
2022-11-30 00:00:00-05:00,342.850006,356.399994,341.480011,356.399994,4251300,0.0,0
2022-12-01 00:00:00-05:00,357.989990,361.579987,354.049988,360.799988,3374700,0.0,0
2022-12-02 00:00:00-05:00,357.450012,362.290009,356.059998,360.059998,2015500,0.0,0


In [7]:
# select only the close column from the DataFrames  
pypl_df = pypl_df[['Close']]
sq_df = sq_df[['Close']]
ma_df = ma_df[['Close']]

# concat close data into one DataFrame
paytech_daily_returns = pd.concat([pypl_df, sq_df, ma_df], axis=1)

# rename columns to each stock ticker instead of just Close
paytech_daily_returns.columns = ['PYPL', 'SQ', 'MA']

display(paytech_daily_returns.head())
display(paytech_daily_returns.tail())

,PYPL,SQ,MA
Date,,,
2017-12-04 00:00:00-05:00,70.970001,36.869999,139.599030
2017-12-05 00:00:00-05:00,71.199997,37.599998,141.253632
2017-12-06 00:00:00-05:00,73.139999,37.610001,143.482483
2017-12-07 00:00:00-05:00,73.690002,38.730000,145.458221
2017-12-08 00:00:00-05:00,72.910004,38.090000,145.886490


,PYPL,SQ,MA
Date,,,
2022-11-28 00:00:00-05:00,79.930000,61.369999,344.589996
2022-11-29 00:00:00-05:00,77.639999,62.110001,343.130005
2022-11-30 00:00:00-05:00,78.410004,67.769997,356.399994
2022-12-01 00:00:00-05:00,78.529999,69.169998,360.799988
2022-12-02 00:00:00-05:00,74.660004,68.180000,360.059998


In [8]:
# convert the closing prices to daily returns 
paytech_daily_returns = paytech_daily_returns.pct_change().dropna()

display(paytech_daily_returns.head())
display(paytech_daily_returns.tail())

,PYPL,SQ,MA
Date,,,
2017-12-05 00:00:00-05:00,0.003241,0.019799,0.011853
2017-12-06 00:00:00-05:00,0.027247,0.000266,0.015779
2017-12-07 00:00:00-05:00,0.007520,0.029779,0.013770
2017-12-08 00:00:00-05:00,-0.010585,-0.016525,0.002944
2017-12-11 00:00:00-05:00,0.005212,0.005513,0.006605


,PYPL,SQ,MA
Date,,,
2022-11-28 00:00:00-05:00,-0.001873,-0.031714,-0.019073
2022-11-29 00:00:00-05:00,-0.028650,0.012058,-0.004237
2022-11-30 00:00:00-05:00,0.009918,0.091129,0.038673
2022-12-01 00:00:00-05:00,0.001530,0.020658,0.012346
2022-12-02 00:00:00-05:00,-0.049280,-0.014313,-0.002051


### Get 5 years of data for Lending sector - LendingTree, LendingClub, FIS (Fidelity National Information Services)

In [9]:
# create ticker object for lending stocks 
lending_tickers = yf.Tickers('tree lc fis')

In [10]:
# create DataFrames for 5 years of data 
tree_df = lending_tickers.tickers['TREE'].history(period='5y')
lc_df = lending_tickers.tickers['LC'].history(period='5y')
fis_df = lending_tickers.tickers['FIS'].history(period='5y')

print('LendingTree DataFrame')
display(tree_df.head())
display(tree_df.tail())

print('LendingClub DataFrame')
display(lc_df.head())
display(lc_df.tail())

print('FIS DataFrame')
display(fis_df.head())
display(fis_df.tail())

LendingTree DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,309.850006,316.250000,301.450012,303.399994,246900,0,0
2017-12-05 00:00:00-05:00,301.950012,313.299988,298.609985,305.200012,185000,0,0
2017-12-06 00:00:00-05:00,305.350006,310.799988,304.200012,306.899994,190300,0,0
2017-12-07 00:00:00-05:00,307.049988,316.600006,305.980011,311.250000,125300,0,0
2017-12-08 00:00:00-05:00,314.950012,319.429993,308.899994,315.049988,258700,0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,23.379999,23.379999,22.200001,22.370001,235900,0,0
2022-11-29 00:00:00-05:00,22.500000,23.459999,22.010000,23.330000,262400,0,0
2022-11-30 00:00:00-05:00,23.200001,23.990000,21.750000,23.950001,376000,0,0
2022-12-01 00:00:00-05:00,24.000000,25.240000,23.170000,25.120001,256900,0,0
2022-12-02 00:00:00-05:00,24.250000,25.110001,23.629999,24.980000,204100,0,0


LendingClub DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,21.950001,22.275000,21.350000,21.850000,1400220,0,0.0
2017-12-05 00:00:00-05:00,21.850000,21.950001,21.250000,21.250000,1117300,0,0.0
2017-12-06 00:00:00-05:00,21.150000,21.400000,20.799999,21.250000,1079140,0,0.0
2017-12-07 00:00:00-05:00,18.549999,19.700001,16.450001,17.950001,9593000,0,0.0
2017-12-08 00:00:00-05:00,18.700001,18.700001,17.299999,17.549999,4030720,0,0.0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,10.37,10.470,9.920,10.06,2000200,0,0.0
2022-11-29 00:00:00-05:00,10.04,10.435,10.040,10.26,859700,0,0.0
2022-11-30 00:00:00-05:00,10.14,10.395,9.930,10.31,1532500,0,0.0
2022-12-01 00:00:00-05:00,10.42,10.550,10.195,10.24,1034500,0,0.0
2022-12-02 00:00:00-05:00,10.19,10.375,9.990,10.18,1111000,0,0.0


FIS DataFrame


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-12-04 00:00:00-05:00,88.692227,88.757910,87.819567,87.857101,1668400,0.0,0
2017-12-05 00:00:00-05:00,88.025992,89.358445,87.547442,89.302139,2017200,0.0,0
2017-12-06 00:00:00-05:00,89.105078,89.649318,88.607757,89.424118,1817100,0.0,0
2017-12-07 00:00:00-05:00,89.546103,90.090344,89.330281,89.958977,2004000,0.0,0
2017-12-08 00:00:00-05:00,90.240486,90.493842,89.621183,89.940216,1683500,0.0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-28 00:00:00-05:00,65.330002,67.169998,65.169998,65.529999,6872700,0.0,0
2022-11-29 00:00:00-05:00,65.699997,68.680000,65.550003,67.529999,8373600,0.0,0
2022-11-30 00:00:00-05:00,67.930000,72.620003,67.709999,72.580002,13852900,0.0,0
2022-12-01 00:00:00-05:00,72.690002,74.300003,72.349998,73.180000,8105200,0.0,0
2022-12-02 00:00:00-05:00,72.750000,75.000000,72.589996,74.059998,8741500,0.0,0


In [11]:
# select only the close column from each
tree_df = tree_df[['Close']]
lc_df = lc_df[['Close']]
fis_df = fis_df[['Close']]

# concat close data into one DataFrame
lending_daily_returns = pd.concat([tree_df, lc_df, fis_df], axis=1)

# rename columns to each stock ticker instead of just Close
lending_daily_returns.columns = ['TREE', 'LC', 'FIS']

display(lending_daily_returns.head())
display(lending_daily_returns.tail())

,TREE,LC,FIS
Date,,,
2017-12-04 00:00:00-05:00,303.399994,21.850000,87.857101
2017-12-05 00:00:00-05:00,305.200012,21.250000,89.302139
2017-12-06 00:00:00-05:00,306.899994,21.250000,89.424118
2017-12-07 00:00:00-05:00,311.250000,17.950001,89.958977
2017-12-08 00:00:00-05:00,315.049988,17.549999,89.940216


,TREE,LC,FIS
Date,,,
2022-11-28 00:00:00-05:00,22.370001,10.06,65.529999
2022-11-29 00:00:00-05:00,23.330000,10.26,67.529999
2022-11-30 00:00:00-05:00,23.950001,10.31,72.580002
2022-12-01 00:00:00-05:00,25.120001,10.24,73.180000
2022-12-02 00:00:00-05:00,24.980000,10.18,74.059998


In [12]:
# convert the closing prices to daily returns 
lending_daily_returns = lending_daily_returns.pct_change().dropna()

display(lending_daily_returns.head())
display(lending_daily_returns.tail())

,TREE,LC,FIS
Date,,,
2017-12-05 00:00:00-05:00,0.005933,-0.027460,0.016448
2017-12-06 00:00:00-05:00,0.005570,0.000000,0.001366
2017-12-07 00:00:00-05:00,0.014174,-0.155294,0.005981
2017-12-08 00:00:00-05:00,0.012209,-0.022284,-0.000209
2017-12-11 00:00:00-05:00,-0.019521,0.108262,-0.001461


,TREE,LC,FIS
Date,,,
2022-11-28 00:00:00-05:00,-0.051717,-0.047348,-0.006820
2022-11-29 00:00:00-05:00,0.042915,0.019881,0.030520
2022-11-30 00:00:00-05:00,0.026575,0.004873,0.074782
2022-12-01 00:00:00-05:00,0.048852,-0.006790,0.008267
2022-12-02 00:00:00-05:00,-0.005573,-0.005859,0.012025


### Get 5 years of data for Wealthtech sector = (stocks TBD)

## Data Analysis and Calculations

# Monte Carlo Simulations